In [152]:
import json
import numpy as np

In [153]:
with open('/Users/juanda/Downloads/emociones.json') as file:
    input_data = json.load(file)

Convierto el formato del json a un dataset, que será un array de objetos de tipo {input: 'el texto', output: 'la etiqueta'}

In [154]:
ds_input_data = []
li_labels = list(emociones.keys())
for label in emociones.keys():
    for text in input_data[label]:
        ds_input_data.append({'input': text, 'output': label})
ar_labels

array(dict_keys(['contento', 'triste']), dtype=object)

In [155]:
from random import shuffle
shuffle(ds_input_data)

In [156]:
import import_ipynb
import bag_of_words as bow

In [157]:
import numpy as np
texts = [entry['input'] for entry in ds_input_data]
labels = [utils.to_categorical(li_labels.index(entry['output']), num_classes=2) for entry in ds_input_data]

dictionary = bow.extractDictionary(texts)

vc_input = np.array([bow.bow(text, dictionary) for text in texts])
vc_labels = np.array(labels)


In [158]:
from keras import models
from keras import layers
from keras import utils


In [159]:
input_size = vc_input.shape[1]
model = models.Sequential()
model.add(layers.Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(layers.Dense(2, activation='relu'))

In [160]:
from keras import optimizers
model.compile(optimizers.RMSprop(lr=0.001), loss='mse', metrics=['accuracy'])
